In [18]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import AerSimulator

def simulate(circuit, simulator=AerSimulator()):
    #execute the circuit on a simulator
    compiled_circuit = transpile(circuit, simulator)
    
    job = simulator.run(compiled_circuit)
    print(f"job: {job}")
    
    result = job.result()
    print(f"result: {result}")

    #get counts of the outcomes
    counts = result.get_counts(circuit)
    print(f"counts: {counts}")

simulator = AerSimulator()


In [ ]:
#create a circuit acting on a quantum register of one qubit
circuit = QuantumCircuit(1, 1)

#apply Hadamard to qubit 0
circuit.h(0)

#measure qubit 0
circuit.measure(0 ,0)
simulate(circuit, simulator)


job: <qiskit_aer.jobs.aerjob.AerJob object at 0x7f37bcf17340>
result: Result(backend_name='aer_simulator', backend_version='0.15.1', qobj_id='', job_id='66a26ec1-b204-432a-9657-93cc7c91029c', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x1': 522, '0x0': 502}), header=QobjExperimentHeader(creg_sizes=[['c', 1]], global_phase=0.0, memory_slots=1, n_qubits=1, name='circuit-184', qreg_sizes=[['q', 1]], metadata={}), status=DONE, seed_simulator=223161732, metadata={'time_taken': 0.00134442, 'num_bind_params': 1, 'parallel_state_update': 1, 'parallel_shots': 1, 'required_memory_mb': 0, 'input_qubit_map': [[0, 0]], 'method': 'stabilizer', 'device': 'CPU', 'num_qubits': 1, 'sample_measure_time': 0.000265207, 'active_input_qubits': [0], 'num_clbits': 1, 'remapped_qubits': False, 'runtime_parameter_bind': False, 'max_memory_mb': 7940, 'noise': 'ideal', 'measure_sampling': True, 'batched_shots_optimization': False, 'fusion': {'

### 12 Coin Flip/Unflip


In [4]:
circuit = QuantumCircuit(1, 1)
circuit.h(0)
circuit.measure(0 ,0)
simulate(circuit, simulator)

circuit = QuantumCircuit(1, 1)
circuit.h(0)
circuit.h(0)
circuit.measure(0 ,0)
simulate(circuit, simulator)


{'1': 510, '0': 514}
{'0': 1024}


### 12 New Coin Flip/Unflip


In [5]:
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.measure_all()
simulate(circuit, simulator)
circuit.measure_all()
simulate(circuit, simulator)
circuit.measure_all()
simulate(circuit, simulator)


{'0': 481, '1': 543}
{'1 1': 513, '0 0': 511}
{'0 0 0': 501, '1 1 1': 523}


In [26]:
from qiskit_aer.primitives import SamplerV2
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

circuit = QuantumCircuit(1)
circuit.h(0)

# initialization of the sampler
sampler = SamplerV2()
print(f"sampler: {sampler}")

circuit.measure_all()
# simulate(circuit, simulator)

# collect 128 shots from the Bell circuit
primitive_job = sampler.run([circuit], shots=128)#.result()
print(f"primitive_job: {primitive_job}")

result = primitive_job.result()
print(f"result: {result}")

# print(result.counts())


sampler: <qiskit_aer.primitives.sampler_v2.SamplerV2 object at 0x7f37bcf42020>
primitive_job: <qiskit.primitives.primitive_job.PrimitiveJob object at 0x7f37bcf42d70>
result: PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=128, num_bits=1>)), metadata={'shots': 128, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 1.3931e-05, 'time_taken_execute': 0.00038295, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 7940, 'parallel_experiments': 1}})], metadata={'version': 2})


In [26]:
from qiskit import transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

sim = AerSimulator()
# --------------------------
# Simulating using estimator
#---------------------------
from qiskit_aer.primitives import EstimatorV2

psi1 = transpile(RealAmplitudes(num_qubits=2, reps=2), sim, optimization_level=0)
psi2 = transpile(RealAmplitudes(num_qubits=2, reps=3), sim, optimization_level=0)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
theta3 = [1, 2, 3, 4, 5, 6]

estimator = EstimatorV2()

# calculate [ [<psi1(theta1)|H1|psi1(theta1)>,
#              <psi1(theta3)|H3|psi1(theta3)>],
#             [<psi2(theta2)|H2|psi2(theta2)>] ]
job = estimator.run(
    [
        (psi1, [H1, H3], [theta1, theta3]),
        (psi2, H2, theta2)
    ],
    precision=0.01
)
result = job.result()
print(f"expectation values : psi1 = {result[0].data.evs}, psi2 = {result[1].data.evs}")

# --------------------------
# Simulating using sampler
# --------------------------
from qiskit_aer.primitives import SamplerV2
from qiskit import QuantumCircuit

# create a Bell circuit
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# create two parameterized circuits
pqc = RealAmplitudes(num_qubits=2, reps=2)
pqc.measure_all()
pqc = transpile(pqc, sim, optimization_level=0)
pqc2 = RealAmplitudes(num_qubits=2, reps=3)
pqc2.measure_all()
pqc2 = transpile(pqc2, sim, optimization_level=0)

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [0, 1, 2, 3, 4, 5, 6, 7]

# initialization of the sampler
sampler = SamplerV2()
print(sampler)

# collect 128 shots from the Bell circuit
job = sampler.run([bell], shots=128)
job_result = job.result()
print(f"counts for Bell circuit : {job_result[0].data.meas.get_counts()}")
 
# run a sampler job on the parameterized circuits
job2 = sampler.run([(pqc, theta1), (pqc2, theta2)])
job_result = job2.result()
print(f"counts for parameterized circuit : {job_result[0].data.meas.get_counts()}")

# --------------------------------------------------
# Simulating with noise model from actual hardware
# --------------------------------------------------
# from qiskit_ibm_runtime import QiskitRuntimeService
# provider = QiskitRuntimeService(channel='ibm_quantum', token="set your own token here")
# backend = provider.get_backend("ibm_kyoto")

# # create sampler from the actual backend
# sampler = SamplerV2.from_backend(backend)

# # run a sampler job on the parameterized circuits with noise model of the actual hardware
# bell_t = transpile(bell, AerSimulator(basis_gates=["ecr", "id", "rz", "sx"]), optimization_level=0)
# job3 = sampler.run([bell_t], shots=128)
# job_result = job3.result()
# print(f"counts for Bell circuit w/noise: {job_result[0].data.meas.get_counts()}")

expectation values : psi1 = [ 1.55961765 -1.09307785], psi2 = 0.1915812368305494
counts for Bell circuit : {'00': 66, '11': 62}
counts for parameterized circuit : {'01': 390, '11': 396, '00': 140, '10': 98}


In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
 
# Step 1: Define operator
op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)
 
# Step 2: Define quantum state
state = QuantumCircuit(2)
state.x(0)
state.x(1)
 
# Define provider and simulator backend
from qiskit_ibm_provider import IBMProvider
 
 
provider = IBMProvider()
 
backend = provider.get_backend("ibmq_qasm_simulator")
 
# Define a statevector simulator
# from qiskit_aer import AerSimulator
# backend = AerSimulator(method="statevector", shots=100)
 
# Define a real backend
# backend = provider.get_backend("ibm_brisbane")
 
# Circuits must obey the Instruction Set Architecture (ISA) of a particular backend.
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
 
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
 
from qiskit.opflow import CircuitStateFn, PauliSumOp
 
opflow_op = PauliSumOp(op)
opflow_state = CircuitStateFn(isa_state)

/tmp/ipykernel_24/3449596947.py:21: DeprecationWarning: The package qiskit_ibm_provider is being deprecated. Please see https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime to get instructions on how to migrate to qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  from qiskit_ibm_provider import IBMProvider


AccountNotFoundError: 'No default ibm_quantum account saved.'